In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

ohe=OneHotEncoder()
lgbm=LGBMClassifier(n_jobs=-1,random_state=42)

#### import / missing treatment 

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train_id=train['ID']
test_id=test['ID']
target='Is_Lead'
train=train.drop(columns=['ID'])
test=test.drop(columns=['ID'])

In [3]:
train1=train.fillna('Others')
test1=test.fillna('Others')

In [4]:
X=train1.drop(columns=target)
y=train1[target]

In [5]:
cat_cols=[i for i in train1.columns if train1[i].dtypes=='object']

In [30]:
ct=ColumnTransformer([('ohe',ohe,cat_cols)],
                    remainder='passthrough',
                    sparse_threshold=0,
                    n_jobs=-1)

#### feature engineering

In [8]:
X['vint_frac']=X['Vintage']/X['Age']
test1['vint_frac']=test1['Vintage']/test1['Age']

In [9]:
X['average_occ_bal']=X.groupby(['Occupation'])['Avg_Account_Balance'].transform('mean')
test1['average_occ_bal']=test1.groupby(['Occupation'])['Avg_Account_Balance'].transform('mean')

In [10]:
X['average_prod_bal']=X.groupby(['Credit_Product'])['Avg_Account_Balance'].transform('mean')
test1['average_prod_bal']=test1.groupby(['Credit_Product'])['Avg_Account_Balance'].transform('mean')

In [119]:
# from imblearn.combine import SMOTEENN

In [120]:
# se=SMOTEENN(random_state=42,n_jobs=-1)

#### to numerical array

In [32]:
X1=ct.fit_transform(X)

In [33]:
test_df=ct.transform(test1)

In [130]:
# X_rs,y_rs=se.fit_resample(X1,y)

In [131]:
# X_rs.shape, y_rs.shape

((199238, 56), (199238,))

#### KFOLD

In [137]:
skf=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

In [132]:
df=pd.DataFrame(columns=range(0,11)) ### dataframe for prediction

In [143]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.072958,0.070920,0.077598,0.103788,0.107373,0.078606,0.087721,0.085063,0.086094,0.089078,NaN
1,0.999970,0.999962,0.999918,0.999949,0.999966,0.999930,0.999955,0.999967,0.999971,0.999957,NaN
2,0.004270,0.802164,0.843931,0.007598,0.849726,0.006460,0.007680,0.825541,0.837329,0.846265,NaN
3,0.004160,0.745466,0.796286,0.003436,0.787577,0.003600,0.004188,0.779667,0.785921,0.765475,NaN
4,0.003820,0.745516,0.750330,0.003454,0.733153,0.003141,0.004123,0.763353,0.789092,0.768028,NaN


In [146]:
df.iloc[0,0:10].sum()/10

0.08591985024323

In [147]:
df.iloc[0].mean()

0.08591985024323

In [140]:
# skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

In [141]:
# def kfoldscore(model):
#     val=0
#     for fold,(train_idx,test_idx) in enumerate(skf.split(X1,y)):
        
#         X_train=X1[train_idx,:]
#         y_train=y.iloc[train_idx]
        
#         X_rs,y_rs=se.fit_resample(X_train,y_train)
    
#         X_test=X1[test_idx,:]
#         y_test=y.iloc[test_idx]
        
#         model.fit(X_rs,y_rs)
        
#         y_pred=model.predict_proba(X_test)[:,1]
        
#         df[fold]=model.predict_proba(test_df)[:,1]
        
#         print(roc_auc_score(y_test,y_pred))
#         val+=roc_auc_score(y_test,y_pred)
        
#     print('\nMean roc auc score is',round(val/10,5))

In [74]:
def kfoldscore(model):
    val=0
    for fold,(train_idx,test_idx) in enumerate(skf.split(X1,y)):
        
        X_train=X1[train_idx,:]
        y_train=y.iloc[train_idx]
    
        X_test=X1[test_idx,:]
        y_test=y.iloc[test_idx]
        
        model.fit(X_train,y_train)
        y_pred=model.predict_proba(X_test)[:,1]
        
        df[fold]=model.predict_proba(test_df)[:,1]
        
        print(roc_auc_score(y_test,y_pred))
        val+=roc_auc_score(y_test,y_pred)
        
    print('\nMean roc auc score is',round(val/10,5))

In [75]:
# kfoldscore(lgbm)

0.8745247292537854
0.8762811739717824
0.8738906108558273
0.8714955675505645
0.8752108709326928
0.8711556217779212
0.8738376971548306
0.8751780993147947
0.87468846950188
0.8748323237450061

Mean roc auc score is 0.8741


In [78]:
lgb_pred1=df.mean(axis=1)

In [79]:
def make_csv_pred(name):
    df=pd.DataFrame({'ID':test_id,target:lgb_pred1})
    df.to_csv(name,index=False)

In [80]:
make_csv_pred('lgbm_v4.csv')